In [2]:
import pandas as pd
df=pd.read_csv("startup data.csv")
#df["is_CA"]

In [3]:
# Drop specified columns
df = df.drop(['Unnamed: 0', 'zip_code', 'id', 'Unnamed: 6', 'labels', 'state_code.1', 'object_id','closed_at','age_last_milestone_year','first_funding_at','last_funding_at','age_first_milestone_year'], axis=1)

# Create a new column 'state' based on the existing state columns
df['state'] = df[['is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate']].idxmax(axis=1)
df['state'] = df['state'].apply(lambda x: x[3:] if x != 'is_otherstate' else 'Other')

# Drop the individual state columnss
df = df.drop(['is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate'], axis=1)

# Create a new column 'Sector' based on the individual sector columns
sectors = ['is_software', 'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 
           'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 'is_othercategory']

df['Sector'] = df[sectors].idxmax(axis=1)

# Mapping to rename sectors
sector_mapping = {
    'is_software': 'software',
    'is_web': 'web',
    'is_mobile': 'mobile',
    'is_enterprise': 'enterprise',
    'is_advertising': 'advertising',
    'is_gamesvideo': 'gamesvideo',
    'is_ecommerce': 'ecommerce',
    'is_biotech': 'biotech',
    'is_consulting': 'consulting',
    'is_othercategory': 'othercategory'
}

# Rename sectors using mapping
df['Sector'] = df['Sector'].replace(sector_mapping)

# Now 'is_software' becomes 'software', 'is_web' becomes 'web', and so on.

# Drop the individual sector columns
df = df.drop(sectors, axis=1)



In [4]:
#cellule pour tester des trucs
df.head()
column_names = df.columns

# Print the column names
print(column_names)

Index(['state_code', 'latitude', 'longitude', 'city', 'name', 'founded_at',
       'age_first_funding_year', 'age_last_funding_year', 'relationships',
       'funding_rounds', 'funding_total_usd', 'milestones', 'category_code',
       'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC',
       'has_roundD', 'avg_participants', 'is_top500', 'status', 'state',
       'Sector'],
      dtype='object')


In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
from plotly import graph_objs as go 
import plotly.graph_objects as go
import numpy as np

# Assuming 'df' is your DataFrame with columns including 'status' and 'category_code'
# Replace this with your actual data

# Sample data for demonstration

# Calculate the proportion of acquired companies in each category
category_proportions = df.groupby('Sector')['status'].apply(lambda x: (x == 'acquired').mean()).reset_index(name='proportion')


df['new_total_funding_usd'] = df['funding_total_usd'].apply(lambda x: 80000000 if x >= 80000000 else x)
#df['funding_total_usd'] = np.sqrt(df['funding_total_usd'])

# Create a Dash app
app = dash.Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
tickvaleurs = [0,4000000,8000000, 12000000,16000000, 20000000, 30000000, 40000000, 50000000,60000000,70000000,80000000]

fig = px.histogram(df,x='new_total_funding_usd',color='status',color_discrete_map={'acquired': '#4831D4', 'closed': '#FF69B4'})

fig.update_xaxes(tickvals=tickvaleurs, ticktext=['0','4M','8M','12M','16M','20M','30M','40M','50M','60M','70M','>80M'],tickmode='array')
fig.update_traces(xbins_size=4000000)
fig.update_xaxes(title_text='Funding Total (USD)')


#change the background of the plot 
"""fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background']
)"""

#fig.update_xaxes(labelalias={'80M - 83.99M':'>80M'})

# Define the layout
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(children='How start-up succeed in America',
            style={'text-align': 'center', 'color': colors['text'], 'font-family': 'Arial, sans-serif'}),

    dbc.Row([
        dbc.Col([dcc.Graph(
            id='bar-chart',
            figure=px.bar(category_proportions, x='Sector', y='proportion',
                     labels={'new_total_funding_usd': 'Funding Total (en USD)'},
                     color='Sector',
                     title='Proportion of Acquired Companies by Sector',
                     ))
        ]),
    
    dbc.Col([
    dcc.Graph(style={'backgroundColor': colors['background']},
        id='histogram',
        figure=fig,)
    
        ])
    
    ])
])
        


app.run_server(debug=True)


In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
from plotly import graph_objs as go 
import plotly.graph_objects as go
import numpy as np
from dash.dependencies import Input, Output

# Assuming 'df' is your DataFrame with columns including 'status' and 'category_code'
# Replace this with your actual data

# Sample data for demonstration

# Calculate the proportion of acquired companies in each category
category_proportions = df.groupby('Sector')['status'].apply(lambda x: (x == 'acquired').mean()).reset_index(name='proportion')


df['new_total_funding_usd'] = df['funding_total_usd'].apply(lambda x: 80000000 if x >= 80000000 else x)
#df['funding_total_usd'] = np.sqrt(df['funding_total_usd'])

# Create a Dash app
app = dash.Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
def histofunding():
    tickvaleurs = [0,4000000,8000000, 12000000,16000000, 20000000, 30000000, 40000000, 50000000,60000000,70000000,80000000]

    fig = px.histogram(df,x='new_total_funding_usd',color='status',color_discrete_map={'acquired': '#4831D4', 'closed': '#FF69B4'})

    fig.update_xaxes(tickvals=tickvaleurs, ticktext=['0','4M','8M','12M','16M','20M','30M','40M','50M','60M','70M','>80M'],tickmode='array')
    fig.update_traces(xbins_size=4000000)
    fig.update_xaxes(title_text='Funding Total (USD)')
    fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'])
    return fig

def historelation():
    fig=px.histogram(df,x='relationships',color='status',color_discrete_map={'acquired': '#4831D4', 'closed': '#FF69B4'})
    fig.update_traces(xbins_size=2)
    return fig


#change the background of the plot 
"""fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background']
)"""

#fig.update_xaxes(labelalias={'80M - 83.99M':'>80M'})

# Define the layout
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(children='How start-up succeed in America',
            style={'text-align': 'center', 'color': colors['text'], 'font-family': 'Arial, sans-serif'}),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id='pie-chart',
                figure=px.pie(df, values='new_total_funding_usd', names='Sector')
            ),
        ], width=3),  # Adjust width as needed
        
        dbc.Col([
            dcc.Graph(
                id='bar-chart',
                figure=px.bar(category_proportions, x='Sector', y='proportion',
                              labels={'new_total_funding_usd': 'Funding Total (en USD)'},
                              color='Sector',
                              title='Proportion of Acquired Companies by Sector',
                              )
            ),
        ], width=9),  # Adjust width as needed
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id="histo-drop",
                options=[
                    {'label': 'Startup total funding', 'value': 'Startup total funding'},
                    {'label': 'Startup relationships', 'value': 'Startup relationships'},
                ], 
                value='Startup total funding',
                style={'width': 220, 'text-align': 'left', 'color': colors['text'], 'font-family': 'Arial, sans-serif'},
            ),
            dcc.Graph(
                style={'backgroundColor': colors['background']},
                id='histogram',
                figure=histofunding(),
            )
        ])
    ])
])



@app.callback(
        Output(component_id='histogram', component_property='figure'), # (1)
        Input(component_id='histo-drop', component_property='value') # (2)
        )
def update_legend(input_value):
    if input_value=='Startup total funding':
        return histofunding()
    elif input_value =='Startup relationships':
        return historelation()
        
        


app.run_server(debug=True)


In [7]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
from plotly import graph_objs as go 
import plotly.graph_objects as go
import numpy as np
from dash.dependencies import Input, Output

# Assuming 'df' is your DataFrame with columns including 'status' and 'category_code'
# Replace this with your actual data

# Sample data for demonstration

# Calculate the proportion of acquired companies in each category
category_proportions = df.groupby('Sector')['status'].apply(lambda x: (x == 'acquired').mean()).reset_index(name='proportion')


df['new_total_funding_usd'] = df['funding_total_usd'].apply(lambda x: 80000000 if x >= 80000000 else x)
#df['funding_total_usd'] = np.sqrt(df['funding_total_usd'])

# Create a Dash app


colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
def histofunding():
    tickvaleurs = [0,4000000,8000000, 12000000,16000000, 20000000, 30000000, 40000000, 50000000,60000000,70000000,80000000]

    fig = px.histogram(df,x='new_total_funding_usd',color='status',color_discrete_map={'acquired': '#4831D4', 'closed': '#FF69B4'})

    fig.update_xaxes(tickvals=tickvaleurs, ticktext=['0','4M','8M','12M','16M','20M','30M','40M','50M','60M','70M','>80M'],tickmode='array')
    fig.update_traces(xbins_size=4000000)
    fig.update_xaxes(title_text='Funding Total (USD)')
    return fig

def historelation():
    fig=px.histogram(df,x='relationships',color='status',color_discrete_map={'acquired': '#4831D4', 'closed': '#FF69B4'})
    fig.update_traces(xbins_size=2)
    return fig


#change the background of the plot 
"""fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background']
)"""

#fig.update_xaxes(labelalias={'80M - 83.99M':'>80M'})

# Define the layout
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container(fluid=True, children=[
    html.H1(children='How start-up succeed in America',
            style={'text-align': 'center', 'color': colors['text'], 'font-family': 'Arial, sans-serif'}),
    
    dbc.Col([
        dcc.Graph(
            id='pie-chart',
            figure=px.pie(df, values='new_total_funding_usd', names='Sector')
            )
        ])
     # This column spans half the width of the row
        dbc.Col([
            html.Div([
                dcc.Dropdown(
                    id="histo-drop",
                    options=[
                        {'label': 'Startup total funding', 'value': 'Startup total funding'},
                        {'label': 'Startup relationships', 'value': 'Startup relationships'},
                    ], 
                    value='Startup total funding',
                    style={'width': 220, 'text-align': 'left', 'color': colors['text'], 'font-family': 'Arial, sans-serif'},
                ),
                dcc.Graph(
                    style={'backgroundColor': colors['background']},
                    id='histogram',
                    figure=histofunding(),
                )
            ])
        ], 
        width=6
        ),
         # This column spans the entire width of the row
    ]),
    
    dbc.Row([
        

        dbc.Col([
            dcc.Graph(
                id='bar-chart',
                figure=px.bar(category_proportions, x='Sector', y='proportion',
                              labels={'new_total_funding_usd': 'Funding Total (en USD)'},
                              color='Sector',
                              title='Proportion of Acquired Companies by Sector',
                          ))
        ],
        width={'size':6,'offset':6}), # This column spans half the width of the row
    ])
])



@app.callback(
        Output(component_id='histogram', component_property='figure'), # (1)
        Input(component_id='histo-drop', component_property='value') # (2)
        )
def update_legend(input_value):
    if input_value=='Startup total funding':
        return histofunding()
    elif input_value =='Startup relationships':
        return historelation()
        
        


app.run_server(debug=True)


SyntaxError: unmatched ']' (4074143628.py, line 100)